In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd "/content/gdrive/MyDrive/CSC 583 Text Retrieval/hatespeech_lexicon_dataset"

/content/gdrive/.shortcut-targets-by-id/1rxvOfGO8GUhjrZK8B0UxXwAOPf-cjgvi/CSC 583 Text Retrieval/hatespeech_lexicon_dataset


In [ ]:
import pandas as pd

# Method 1: Build Lexicon By Intersecting Hate Speech Lexicon

In [ ]:
# load Davidson's hate speech lexicon 
hatespeech1 = pd.read_csv("./davidson_refined_ngram_dict.csv")
hatespeech1

,ngram,prophate
0,allah akbar,0.870
1,blacks,0.583
2,chink,0.467
3,chinks,0.542
4,dykes,0.602
...,...,...
173,nigga you a lame,0.556
174,niggers are in my,0.714
175,wit a lame nigga,0.556
176,you a lame bitch,0.556


In [ ]:
hatespeech1_lexicon = hatespeech1['ngram'].values.tolist()

In [ ]:
# load second hate speech lexicon data
hatespeech2 = pd.read_csv("./hurtlex_EN.tsv", sep='\t',error_bad_lines=False)
hatespeech2

<ipython-input-66-f912213db960>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  hatespeech2 = pd.read_csv("./hurtlex_EN.tsv", sep='\t',error_bad_lines=False)


,id,pos,category,stereotype,lemma,level
0,EN1382,n,qas,no,gag reel,inclusive
1,EN7077,a,cds,no,snotty,conservative
2,EN6856,n,is,yes,mendicant,conservative
3,EN5485,n,re,no,maffias,conservative
4,EN5024,n,cds,no,lying in trade,conservative
...,...,...,...,...,...,...
8223,EN7899,n,an,no,perosuchus,inclusive
8224,EN1319,n,cds,no,mother fucka,conservative
8225,EN2597,n,cds,no,refuse,conservative
8226,EN521,n,asm,no,simple,inclusive


In [ ]:
hatespeech2_lexicon = hatespeech2['lemma'].values.tolist()

In [ ]:
# get the intersection
intersection = set() 
for phrase in hatespeech1_lexicon:
  if phrase in hatespeech2_lexicon:
    print(phrase)
    intersection.add(phrase)
  # if phrase, check if a word in the phrase is in intersection
  words = phrase.split(" ")
  if len(words) > 2:
    for word in words:
      if word in hatespeech2_lexicon:
        #print("part of word intersection: ", word)
        intersection.add(word)

faggot
homo
nigger
niggers
queers
raped


In [ ]:
print(len(intersection))
print(intersection)

19
{'do', 'dirty', 'fucking', 'homo', 'nigga', 'fuckin', 'queers', 'fuck', 'raped', 'bitch', 'cunt', 'fag', 'niggers', 'queer', 'lame', 'trash', 'faggot', 'nigger', 'white'}


In [ ]:
intersection.remove("do")
intersection.remove("white")

In [ ]:
len(intersection)

17

In [ ]:
intersection # hate speech lexicon 

{'bitch',
 'cunt',
 'dirty',
 'fag',
 'faggot',
 'fuck',
 'fuckin',
 'fucking',
 'homo',
 'lame',
 'nigga',
 'nigger',
 'niggers',
 'queer',
 'queers',
 'raped',
 'trash'}

In [ ]:
# save this as file
f = open("./hatespeech_lexicon.txt", "w")
for word in intersection:
  f.write(word+"\n")
f.close()

# Method 2: Expand the intersection lexicon

In [ ]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

In [ ]:
# download the model and return as object ready for use
model_glove_twitter = api.load("glove-twitter-25") # can replace the dimension and model

In [ ]:
model_glove_twitter.most_similar("policies",topn=10)

[('policy', 0.9484812617301941),
 ('reforms', 0.940393328666687),
 ('laws', 0.9401204586029053),
 ('government', 0.9230710864067078),
 ('regulations', 0.916893482208252),
 ('economy', 0.9110006093978882),
 ('immigration', 0.9105910658836365),
 ('legislation', 0.9089651107788086),
 ('govt', 0.9054745435714722),
 ('regulation', 0.9050778746604919)]

In [ ]:
expanded_intersection = set(intersection)

In [ ]:
for hate_word in intersection:
  expanded = model_glove_twitter.most_similar(hate_word,topn=5)
  for word, similarity in expanded:
    expanded_intersection.add(word)
    print(word)
  print(f"{hate_word} - most similar: {model_glove_twitter.most_similar(hate_word,topn=5)}")

ass
nasty
dick
ratchet
trash
dirty - most similar: [('ass', 0.9531222581863403), ('nasty', 0.9366165995597839), ('dick', 0.9346200823783875), ('ratchet', 0.9336236119270325), ('trash', 0.9310711026191711)]
damn
dude
bad
shit
fuckin
fucking - most similar: [('damn', 0.943954586982727), ('dude', 0.9413571357727051), ('bad', 0.9398874640464783), ('shit', 0.9395081996917725), ('fuckin', 0.9372732639312744)]
nigger
hoe
gangster
idiot
hoer
homo - most similar: [('nigger', 0.7869670987129211), ('hoe', 0.7857903838157654), ('gangster', 0.7611382007598877), ('idiot', 0.7478660941123962), ('hoer', 0.7402228713035583)]
lil
bitch
mf
bruh
dawg
nigga - most similar: [('lil', 0.9541468024253845), ('bitch', 0.9520375728607178), ('mf', 0.9506526589393616), ('bruh', 0.9500958919525146), ('dawg', 0.948303759098053)]
damn
shit
mad
nigga
ass
fuckin - most similar: [('damn', 0.952354907989502), ('shit', 0.947651207447052), ('mad', 0.9470900893211365), ('nigga', 0.9452493786811829), ('ass', 0.937578201293945

In [ ]:
print(expanded_intersection)
print("after expansion size ", len(expanded_intersection))

{'hoe', 'bastard', 'dick', 'asshole', 'dirty', 'heads', 'homo', 'weirdo', 'queers', 'fuck', 'damn', 'ratchet', 'clowns', 'raped', 'dumb', 'nasty', 'bitch', 'clown', 'lowkey', 'wannabe', 'feminine', 'gangsters', 'fag', 'dumped', 'whore', 'hypebeasts', 'prick', 'dumbass', 'homos', 'beaners', 'attacked', 'trash', 'twat', 'faggot', 'nigger', 'maggot', 'hell', 'stabbed', 'lil', 'chick', 'feminist', 'fucking', 'gangster', 'skanks', 'nigga', 'fuckin', 'ass', 'dawg', 'mfs', 'err', 'hoer', 'rats', 'bruh', 'goon', 'dude', 'cunt', 'bad', 'fucker', 'scumbags', 'thot', 'kidnapped', 'mf', 'niggers', 'shit', 'queer', 'lmao', 'murdered', 'straight', 'idiot', 'manners', 'lame', 'faggets', 'fags', 'mad', 'dickhead', 'garbage'}
after expansion size  76


In [ ]:
# save this as file
f = open("./expanded_hatespeech_lexicon.txt", "w")
for word in expanded_intersection:
  f.write(word+"\n")
f.close()